## Print the Names that need to be checked in Bloomberg

In [1]:
import xlwings as xw
import pandas as pd
import openpyxl
import numpy as np
import datetime as datetime

Today = str(datetime.datetime.today().strftime("%Y-%m-%d"))

wb = openpyxl.load_workbook(r'\\S64A603\FCNordic\Structured Equity Trading\Volatility Monitor\Daily category 3 control\IVMvA '+Today+'.xlsm')
ws = wb.active

List = []

for i in wb.worksheets[10:]:
    if i['B14'].value != 'Hist vol scaling':
        for col in i['F2:ER2']:
            for cell in col:
                if (cell.value > 0.90) & (cell.value < 1.15):
                    for row in i['F1:F100']:
                        for c in row:

                             if c.value == 'Difference':
                                    Q = c.row
                             if c.value == 'Relative diff':
                                    Z = c.row
                             if c.value == 'Spread':
                                    X = c.row

                                    for t in i['D'+str(Q)+':'+str(Z)]:
                                        for gg in t:
                                            if gg.value > 0.30:
                                                TT = gg.row

                                                for ll in i[cell.column][TT:Z]:
                                                    if ll.value >= 0.01:
                                                        for t in i['D'+str(Z)+':'+str(X)]:
                                                            for gg in t:
                                                                if gg.value > 0.30:
                                                                    TT = gg.row
                                                                    for ll in i[cell.column][TT:X]:
                                                                        if ll.value >= 1.0:
                                                                            List.append(i)
                                                                        if (ll.value <= -1.0) & (ll.value > -100):
                                                                            List.append(i)

                                                    if (ll.value <= -0.01) & (ll.value != 0) & (ll.value > -100):          
                                                        for t in i['D'+str(Z)+':'+str(X)]:
                                                            for gg in t:
                                                                if gg.value > 0.30:
                                                                    TT = gg.row
                                                                    for ll in i[cell.column][TT:X]:
                                                                        if ll.value >= 1.0:
                                                                            List.append(i)
                                                                        if (ll.value <= -1.0) & (ll.value > -100):
                                                                            List.append(i)

                                                        
print("Total Names in Today's file = "+str(len(wb.get_sheet_names()[10:]))+"\n"+"Check the following names in Bloomberg:")                                                       
W = np.unique(List)

for i in W:
    print(i)

Total Names in Today's file = 46
Check the following names in Bloomberg:
<Worksheet "HSBA">
<Worksheet "SREN">
<Worksheet "ALVGY">
<Worksheet "ROG">
<Worksheet "F">
<Worksheet "SWS1V">
<Worksheet "CA">


### Get their Bloomberg Ticker

In [3]:
from marvin import marvin_cfg

Q = []
for i in W:
    Q.append(i.title)

conn1 = marvin_cfg.get_connection('trmp')

for i in range(0,len(Q)):
    try:
        print(pd.read_sql("Select name, description, bloomberg_ticker from STED.Q_EQ_CONVENTIONS_EXT where name ='"+Q[i]+"'",conn1))
    except:
        pass

   name        description bloomberg_ticker
0  HSBA  HSBC Holdings PLC          HSBA LN
   name  description bloomberg_ticker
0  SREN  SWISS RE AG          SREN SE
    name description bloomberg_ticker
0  ALVGY  Allianz SE           ALV GY
  name       description bloomberg_ticker
0  ROG  Roche Holding AG           ROG SE
  name    description bloomberg_ticker
0    F  Ford Motor Co             F UN
    name description bloomberg_ticker
0  SWS1V  Sanoma OYJ         SAA1V FH
  name   description bloomberg_ticker
0   CA  Carrefour SA            CA FP


### Only show the relevant columns and rows in Inf vol monitor in order to avoid scrolling

In [4]:
import datetime as datetime
import openpyxl

Today = str(datetime.datetime.today().strftime("%Y-%m-%d"))

wb = openpyxl.load_workbook(r'\\S64A603\FCNordic\Structured Equity Trading\Volatility Monitor\Daily category 3 control\IVMvA '+Today+'.xlsm')
ws = wb.active

for i in wb.worksheets[8:]:
    i.freeze_panes = 'F1'
    
for i in wb.worksheets[8:]:
    for col in i['F2:ER2']:
        for cell in col:
            if (cell.value < 0.90) | (cell.value > 1.15):
                i.column_dimensions[cell.column].visible = False
                             
for b in wb.worksheets[8:]:
    for row in b['D5:D100']:
        for cell in row:
             if (cell.value < 0.30) & (cell.value > 0.0001):
                b.row_dimensions[cell.row].hidden = True                

wb.save(r'H:\My documents\freezeExample.xlsx')    

len(wb.get_sheet_names()[8:])

48